# সেশন ২ – RAG মূল্যায়ন রাগাস ব্যবহার করে

রাগাস মেট্রিকস ব্যবহার করে ন্যূনতম RAG পাইপলাইন মূল্যায়ন করুন: উত্তর প্রাসঙ্গিকতা, বিশ্বাসযোগ্যতা, প্রসঙ্গ নির্ভুলতা।


# পরিস্থিতি
এই পরিস্থিতি একটি ক্ষুদ্র Retrieval Augmented Generation (RAG) পাইপলাইন স্থানীয়ভাবে মূল্যায়ন করে। আমরা:
- একটি ছোট সিন্থেটিক ডকুমেন্ট কর্পাস সংজ্ঞায়িত করি।
- ডকুমেন্টগুলিকে এম্বেড করি এবং একটি সাধারণ সাদামাটা সিমিলারিটি রিট্রিভার বাস্তবায়ন করি।
- একটি স্থানীয় মডেল (Foundry Local / OpenAI-সামঞ্জস্যপূর্ণ) ব্যবহার করে ভিত্তিপ্রস্তরিত উত্তর তৈরি করি।
- ragas মেট্রিক্স (`answer_relevancy`, `faithfulness`, `context_precision`) গণনা করি।
- একটি দ্রুত মোড (env `RAG_FAST=1`) সমর্থন করি যা দ্রুত পুনরাবৃত্তির জন্য শুধুমাত্র উত্তর প্রাসঙ্গিকতা গণনা করে।

এই নোটবুকটি ব্যবহার করুন আপনার স্থানীয় মডেল + এম্বেডিং স্ট্যাক বড় কর্পাসে স্কেল করার আগে তথ্যভিত্তিক ভিত্তিপ্রস্তরিত উত্তর তৈরি করে কিনা তা যাচাই করতে।


### ব্যাখ্যা: নির্ভরতা ইনস্টলেশন
প্রয়োজনীয় লাইব্রেরি ইনস্টল করা:
- `foundry-local-sdk` স্থানীয় মডেল ব্যবস্থাপনার জন্য।
- `openai` ক্লায়েন্ট ইন্টারফেসের জন্য।
- `sentence-transformers` ঘন এম্বেডিংসের জন্য।
- `ragas` + `datasets` মূল্যায়ন ও মেট্রিক গণনার জন্য।
- `langchain-openai` রাগাস LLM ইন্টারফেসের অ্যাডাপ্টারের জন্য।

পুনরায় চালানো নিরাপদ; যদি পরিবেশ ইতিমধ্যেই প্রস্তুত থাকে তবে এড়িয়ে যান।


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### ব্যাখ্যা: মূল ইমপোর্ট এবং মেট্রিক্স
মূল লাইব্রেরি এবং রাগাস মেট্রিক্স লোড করা হয়। প্রধান উপাদানগুলো:
- এম্বেডিংয়ের জন্য SentenceTransformer।
- `evaluate` এবং নির্বাচিত রাগাস মেট্রিক্স।
- মূল্যায়ন কর্পাস তৈরি করার জন্য `Dataset`।

এই ইমপোর্টগুলো কোনো রিমোট কল ট্রিগার করে না (এম্বেডিংয়ের জন্য সম্ভাব্য মডেল ক্যাশ লোড ছাড়া)।


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### ব্যাখ্যা: টয় কর্পাস এবং প্রশ্নোত্তর গ্রাউন্ড ট্রুথ
একটি ছোট ইন-মেমরি কর্পাস (`DOCS`), ব্যবহারকারীর প্রশ্নের একটি সেট এবং প্রত্যাশিত গ্রাউন্ড ট্রুথ উত্তর সংজ্ঞায়িত করে। এগুলো দ্রুত এবং নির্ধারিত মেট্রিক গণনার সুযোগ দেয়, যেখানে বাইরের ডেটা আনার প্রয়োজন হয় না। বাস্তব পরিস্থিতিতে, আপনি প্রোডাকশন কোয়েরি এবং কিউরেটেড উত্তরের নমুনা সংগ্রহ করবেন।


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### ব্যাখ্যা: সার্ভিস ইনিট, এম্বেডিংস এবং সেফটি প্যাচ
Foundry Local ম্যানেজার ইনিশিয়ালাইজ করে, `promptTemplate` এর জন্য একটি স্কিমা-ড্রিফট সেফটি প্যাচ প্রয়োগ করে, মডেল আইডি রিজলভ করে, OpenAI-সামঞ্জস্যপূর্ণ ক্লায়েন্ট তৈরি করে এবং ডকুমেন্ট কর্পাসের জন্য ডেন্স এম্বেডিংস প্রি-কম্পিউট করে। এটি রিট্রিভাল + জেনারেশনের জন্য পুনরায় ব্যবহারযোগ্য স্টেট প্রস্তুত করে।


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### ব্যাখ্যা: Retriever Function
একটি সাধারণ ভেক্টর সাদৃশ্য রিট্রিভার সংজ্ঞায়িত করে যা নরমালাইজড এমবেডিংসের উপর ডট প্রোডাক্ট ব্যবহার করে। শীর্ষ-k ডকুমেন্ট (ডিফল্ট k=2) ফেরত দেয়। প্রোডাকশনে স্কেল এবং লেটেন্সির জন্য ANN ইনডেক্স (FAISS, Chroma, Milvus) দিয়ে এটি পরিবর্তন করুন।


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### ব্যাখ্যা: Generation Function
`generate` একটি সীমাবদ্ধ প্রম্পট তৈরি করে (সিস্টেম নির্দেশ দেয় শুধুমাত্র প্রসঙ্গ ব্যবহার করতে) এবং স্থানীয় মডেলকে কল করে। কম তাপমাত্রা (0.1) সৃজনশীলতার চেয়ে সঠিক তথ্য উত্তোলনকে প্রাধান্য দেয়। এটি ছাঁটাই করা উত্তর পাঠ্য ফেরত দেয়।


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### ব্যাখ্যা: ফলব্যাক ক্লায়েন্ট ইনিশিয়ালাইজেশন
নিশ্চিত করে যে `client` বিদ্যমান থাকে, এমনকি যদি পূর্বের ইনিশিয়ালাইজেশন সেলটি এড়িয়ে যাওয়া হয় বা ব্যর্থ হয়—পরে মূল্যায়ন ধাপে NameError প্রতিরোধ করে।


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### ব্যাখ্যা: মূল্যায়ন লুপ এবং মেট্রিকস
মূল্যায়ন ডেটাসেট তৈরি করে (প্রয়োজনীয় কলাম: প্রশ্ন, উত্তর, প্রসঙ্গ, গ্রাউন্ড ট্রুথ, রেফারেন্স) এবং তারপর নির্বাচিত রাগাস মেট্রিকসগুলোর উপর পুনরাবৃত্তি করে।

অপ্টিমাইজেশন:
- FAST_MODE শুধুমাত্র উত্তর প্রাসঙ্গিকতায় সীমাবদ্ধ থাকে দ্রুত ধোঁয়া পরীক্ষার জন্য।
- প্রতি-মেট্রিক লুপ সম্পূর্ণ পুনরায় গণনা এড়ায় যখন একটি মেট্রিক ব্যর্থ হয়।

একটি ডিকশনারি আউটপুট করে যেখানে মেট্রিক -> স্কোর (ব্যর্থ হলে NaN)।


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
